In [401]:
#export necessary packages
import pandas as pd
import numpy as np
import re
import glob
#set the option of dataframe format
pd.set_option('display.max_rows', None)
pd.options.display.max_colwidth = 2000 

In [402]:
###
#open txt files saved in one folder, perform all steps and then append the final dataframes together
dfList = []
fmask = '/home/mila/*.txt*' 
for f in glob.glob(fmask): 
    with open(f) as file:
        df1 = file.read()
###
    
###
#prepare main text - first column in the dataframe
    #find all words in capital letters - this is the beginning of every ad
    words = re.findall(r"([A-Z]+\s?[A-Z]+[^a-z0-9\W])",df1)
    #divide the ads by the capital letter words
    regex = r"\b(?:{})\b".format("|".join(words))
    div = re.split(regex, df1)
    div.pop(0)
    #merge the capital letter words with the rest to get the final text
    text = [i + ' ' + j for i, j in zip(words,div)] 
    #if two capital letter words were divided into 2 ads, merge them together
    for i in range(0,len(text)):
        if len(text[i].split()) < 3:
            text[i+1] = text[i] + ' ' + text[i+1]
###

###
#extract necessary information by columns 
    #create empty arrays to fill them further
    all_capital = [None] * len(div)
    first_capital = [None] * len(div)
    price = [None] * len(div)
    hab = [None] * len(div)
    entrada = [None] * len(div)
    convenir = [None] * len(div)
    #extract the information by every row
    for i in range(0,len(div)):
        #find all capital letter words
        all_capital[i] = str(re.findall(r"([A-Z]+\s?[A-Z]+[^a-z0-9\W])", text[i]))
        #find all words that start with capital letter
        first_capital[i] = str(re.findall(r'[A-Z](?:[A-Z]*(?![a-z])|[a-z]*)', div[i]))
        #extract info on the prices introducing all possible combinations of symbols that can be presented in the text
        price[i] = str(re.findall(r'\d{1}\.\d{3}\.\d{3}|\d{1}[^\S]\.\d{3}\.\d{3}|\d{1}\.\d{3}[^\S]\d{3}|\d{1}\,\d{3}\.\d{3}|\d{1}[^\S]\d{3}\.\d{3}|\d{4}\.\d{3}|\d{1}\.\d{3}\,\d{3}|\d{1}\,\d{3}[^\S]\d{3}|\d{3}\.\d{3}|\d{3}\,\d{3}|\.\d{3}\.\d{3}|\d{3}\.\d{3}|\d{2}\.\d{3}\.\d{3}', div[i]))
        #separate entrada and convnenir if two prices are given
        price[i] = price[i].replace(' .', '.')
        if (len(price[i].split()) == 2) & (', ' in price[i]):
            entrada[i],convenir[i] = price[i].split(', ')
        entrada[i] = str(entrada[i]).replace('None', '')
        convenir[i] = str(convenir[i]).replace('None', '')
        #extract info on the rooms introducing possible combinations of symbols
        hab[i] = str(re.findall(r'\d[^\S]hab|\d[^\S]h\,|\dhab|\d[^\S]hob|\d[^\S]h\.|\d[^\S]hsb\.', div[i]))
        #remove letters leaving only the number
        hab[i] = hab[i].split("h")[0]
###

##
#combine to one dataframe and remove unnecessary characters
    df2 = pd.DataFrame(list(zip(text, all_capital, first_capital,price,entrada,convenir,hab)), columns =['text', 'all_capital','first_capital','price','entrada','convenir','hab']) 
    df2 = df2.replace('\'','', regex=True) 
    df2 = df2.replace('\[','', regex=True) 
    df2 = df2.replace('\]','', regex=True)
###
    
###
#append dataframes
    #add the date column by the name of the text file
    df2['date'] = f
    #append dataframes
    dfList.append(df2)
data = pd.concat(dfList)
#clean date column
data["date"] = data["date"].str.split("mila", expand=True)[1]
###

data

,text,all_capital,first_capital,price,entrada,convenir,hab,date
0,"PADRE CLARET MraU hab., com.000. aseo.1 .800.000 a 5000anir. :i 74208.",PADRE CLARET,"Mra, U",1.800.000,,,,/1980.01.01.txt
1,CONTADO compro piso vmjs.T. 3233411. ¿,CONTADO,T,,,,,/1980.01.01.txt
2,"QUIERE VENDEP su pisO? Agonte le la PopIedod omsbiliaria. oficinas Viladomal, 1 1 6, ando, Junto Granvia. T.3233590.",QUIERE VENDEP,"O, Agonte, Pop, Iedod, Viladomal, Junto, Granvia, T",,,,,/1980.01.01.txt
3,"SANTA EULALIA , 3 hab.. 110.metrO. 1 .600.000. T. 3233411.",SANTA EULALIA,"O, T",1.600.000,,,3,/1980.01.01.txt
4,ORAL .,ORAL,,,,,,/1980.01.01.txt
5,"ORAL . MITRE . amoebi , 1hab., 80r9e. ?700.000. T. 3233590.","ORAL, MITRE",T,700.000,,,1,/1980.01.01.txt
6,AGTE .,AGTE,,,,,,/1980.01.01.txt
7,"AGTE . PROPIEDAD lomo biliaria, le venderá su piso en48 horas. 0tiçnas Viladomat, 1 16. Junto l3ranoia.","AGTE, PROPIEDAD","Viladomat, Junto",,,,,/1980.01.01.txt
8,"CORNELLA , perticular vende piso. Cerco mamado y metro1.600.000. Tel. 332-52-97 y 325-89-89. p_ vetrde piaS asna alta asir sol.T 2034528, tardes.",CORNELLA,"Cerco, Tel, S, T",1.600.000,,,,/1980.01.01.txt
9,VIA .,VIA,,,,,,/1980.01.01.txt


In [403]:
#drop observations without price and number of rooms
data['price'].replace('', np.nan, inplace=True)
data['hab'].replace('', np.nan, inplace=True)
data.dropna(subset=['price'], inplace=True)
data.dropna(subset=['hab'], inplace=True)
data.reset_index(drop=True)

,text,all_capital,first_capital,price,entrada,convenir,hab,date
0,"SANTA EULALIA , 3 hab.. 110.metrO. 1 .600.000. T. 3233411.",SANTA EULALIA,"O, T",1.600.000,,,3,/1980.01.01.txt
1,"ORAL . MITRE . amoebi , 1hab., 80r9e. ?700.000. T. 3233590.","ORAL, MITRE",T,700.000,,,1,/1980.01.01.txt
2,"MON Joeqoin Costo itn. Cer-.toen, 4 h, antiguo, 1,800.000 2180944 - 4b.",MON,"Joeqoin, Costo, Cer","1,800.000",,,4,/1980.01.01.txt
3,"MON - O Adrián, dlin 2 hab., e 00500, 600.000 contado.2180946.",MON,"O, Adri",600.000,,,2,/1980.01.01.txt
4,"MON - ESPIU9UOO. enbreáttcoestrenar, 3 hab., 40 m. taO 3.200.000 000venir 2180944.","MON, ESPIU, UOO","ESPIU, UOO, O",3.200.000,,,3,/1980.01.01.txt
5,ESPIU - Casteildefelo nuevo. 3hab.. 1.100.000 2180946.,ESPIU,Casteildefelo,1.100.000,,,3,/1980.01.01.txt
6,"UOO - Vllaoer de Mar Atico 3 hab,’ pl. park., pivoine. itocarrntnra 3.600 000 2180944.",UOO,"Vllaoer, Mar, Atico",3.600 000,,,3,/1980.01.01.txt
7,"MON Snpulvede - Viladoman75 m2 3 h.. c’nin., cnt,, b., pu. quet anterior. o. nuevo2.500 000 facIlidades 2180944.",MON,"Snpulvede, Viladoman, Ilidades",2.500 000,,,3,/1980.01.01.txt
8,"POR AUSSNTARME 3 hab. o. o, baVo, 14.500 pta5 mes vi dan300.000 prao. única entrada, a loo 15 años quede ouy Infor Oración eGo jueves de 8 a 9noche alt UFIF15A, u, Pelayo, 12. j,e A. Barcelona.","POR AUSSNTARME, UFIF","Vo, Infor, Oraci, Go, UFIF, A, Pelayo, A, Barcelona",300.000,,,3,/1980.01.01.txt
9,"UFIF dvpleo can terreros 400tn.2 piscina en tarnoru superior. Aire ucondicionedo, 3beñes, 2 asaco conlyletos, 5 hobitacionso m9o hab. servicio, 2 salsees es Tren. de LeoCorto, frente el Nou Cariip. Precio 12.500.000 poas. non facilidñdes. También alguilarla.T. 3397671 y 33970(6.",UFIF,"Aire, Tren, Leo, Corto, Nou, Cariip, Precio, Tambi, T",12.500.000,,,5,/1980.01.01.txt


In [118]:
#save in excel
data.to_excel('Barc.xlsx')